# Realizo la clasificacion usando Bernoulli Naive Bayes

## Tabla de resultados

## Importo librerias necesarias

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
#Bernoulli Naive Bayes
from sklearn.naive_bayes import BernoulliNB
#Standard Scaler
from  sklearn.preprocessing  import StandardScaler
#PCA
from sklearn.decomposition import PCA

## Leo todos los datos

In [2]:
df = pd.read_csv('../../data/adri_data/entrenamiento_postulaciones_vistas_contenido.csv', encoding='utf-8')
df.head()

,idaviso,esta_online,contenido,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa,...,hora,dayweek,valor_educacion,sexo,edad,idnombre_area_pref,idnivel_laboral_pref,idtipo_de_trabajo_pref,se_postulo,idpostulante_int
0,1112205158,1,7936048507248058387,0,0,0,1,1,14,480,...,0,2,5,2,23,182,1,0,0,417128
1,1112426744,1,8228217713844563040,0,0,0,0,0,163,6,...,6,1,10,1,28,2,0,0,1,43354
2,1112223876,1,1152927457504920128,0,0,0,0,0,29,71,...,6,5,11,2,31,182,0,0,1,277641
3,1112292963,0,6630320097792098328,0,0,0,0,0,158,1726,...,15,4,6,2,27,142,3,0,1,284234
4,1112495426,0,8103142363029325856,0,0,0,0,0,182,1737,...,18,0,-1,-1,0,-1,-1,-1,0,332567


In [3]:
cols = df.head(0)
cols.drop([
    'idaviso',
    'esta_online',
    'contenido',
    'idzona',
    'idciudad',
    'idmapacalle',
    'idnivel_laboral',
    'iddenominacion_empresa',
    'idpostulante',
    'idnombre_area_pref',
    'idnivel_laboral_pref',
    'idtipo_de_trabajo_pref',
    'se_postulo',
    'idpostulante_int','idtipo_de_trabajo', 'idnombre_area','valor_educacion','hora', 'dayweek', 'edad'],inplace=True, axis=1)
cols = cols.columns

/home/sebastian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [4]:
cols
#'idtipo_de_trabajo', 'idnombre_area', 'hora', 'dayweek','valor_educacion', 'edad'

Index(['sexo'], dtype='object')

In [5]:
X = np.array(df.loc[:,cols])
y = np.array(df['se_postulo']) 

In [6]:
X

array([[2],
       [1],
       [2],
       ...,
       [1],
       [1],
       [2]])

In [7]:
df.groupby('sexo')['sexo'].count()

sexo
-1     185964
 1    5207251
 2    4546893
Name: sexo, dtype: int64

## Aplico Scaler

In [8]:
scaler = StandardScaler() 
scaler.fit(X)
X = scaler.transform(X) 

/home/sebastian/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Realizo el Train Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7506)

## Defino el clasificador

In [11]:
clf = BernoulliNB()

## Entreno el clasificador

In [12]:
clf.fit(X_train, y_train);

## Reviso el score

In [13]:
clf.score(X_test, y_test)

0.6224226894873397

## Predigo con todos los datos

In [14]:
X_full_data = X
pred_full_data = clf.predict(X_full_data)
df['sepostulo'] = pred_full_data

## Armo .csv para subir a Kaggle

In [15]:
top100_full = pd.read_csv("../../test_final_100k.csv", encoding='utf-8')
top100_full = pd.merge(top100_full, df, how='left', on=['idaviso','idpostulante'])
top100_full = top100_full[['id', 'se_postulo', 'sepostulo']]
top100_full.fillna(0, inplace=True)
top100_full['se_postulo'] = top100_full['se_postulo'].astype('int')
top100_full['sepostulo'] = top100_full['sepostulo'].astype('int')
top100_full[(top100_full['sepostulo'] == 0) & (top100_full['se_postulo'] == 1)]['sepostulo']
top100_full.drop('se_postulo', axis=1, inplace=True)
top100_full.to_csv('../../data/final_data/top100k.csv', encoding='utf-8', index=False)